In [115]:
# calculate the log2 fold change and p-value for each (cell line, treatment) pair
import scipy.stats as stats
import pandas as pd
import numpy as np
df = pd.read_csv("231214_cytoplasm_all.csv")
grouped_df = pd.read_csv("grouped_unnorm.csv")


In [116]:
# get column names for the new df
measure_names = []
for col in grouped_df.columns:
    if '_mean' in col:
        measure_name = col.split('_mean')[0]
        measure_names.append(measure_name)
columns = grouped_df.columns[1:4].tolist()
columns += [m + "_log2fc" for m in measure_names]
columns += [m + "_pvalue" for m in measure_names]
columns += [m + "_pvadj" for m in measure_names]

In [117]:
# construct df for volcano plot
# filter out control conditions
filtered_df = grouped_df[~grouped_df['treatment'].str.contains("control|C-09")]
volcano_df = pd.DataFrame(columns=columns)
for col in columns[:3]:
    volcano_df[col] = filtered_df[col]
for col in columns[3:]:
    volcano_df[col] = np.nan

In [118]:
# calculate log2 fold change for every measure
import math
for i, row in volcano_df.iterrows():
    treatment = row['treatment']
    cell_line = row['cell_line']
    experiment = row['experiment']
    match = grouped_df[(grouped_df['treatment'] == treatment) & (grouped_df['cell_line'] == cell_line) & (grouped_df['experiment'] == experiment)]
    # match the control row
    if experiment == 'confluency':
        control_match = grouped_df[(grouped_df['treatment'] == 'C-09') & (grouped_df['cell_line'] == cell_line) & (grouped_df['experiment'] == experiment)]
    else:
        control_match = grouped_df[(grouped_df['treatment'] == '0-control') & (grouped_df['cell_line'] == cell_line) & (grouped_df['experiment'] == experiment)]
    
    if len(control_match) != 1 or len(match) != 1:
        print(f"error at {i}")
    for measure in measure_names:
        volcano_df.at[i, measure + "_log2fc"] = math.log2(match[measure + "_mean"].iloc[0] / control_match[measure + "_mean"].iloc[0])

In [119]:
experiments_treatment_dict = {name: group["treatment"].unique().tolist() for name, group in filtered_df.groupby("experiment")}
experiments_treatment_dict

{'TMRE': ['cccp', 'oligomycin'],
 'confluency': ['C-04', 'C-05', 'C-06', 'C-07', 'C-08', 'C-10'],
 'duroquinone': ['duroquinone',
  'duroquinone_rotenone',
  'ethanol',
  'rotenone',
  'ethanol_rotenone'],
 'etc': ['Cyanide', 'Malonate', 'antimycin a', 'oligomycin', 'rotenone'],
 'glucose': ['high glucose', 'low glucose', 'no glucose'],
 'glytcafao': ['2DG',
  'etomoxir',
  'iodoacetic acid',
  'sodium arsenite',
  'sodium fluoroacetate'],
 'pH': ['5.5', '6.5', '7.5', '8.5', '9.5', '6', '6.8', '7.6', '6.4', '9'],
 'seahorse': ['fccp',
  'oligomycin',
  'rotenone_antimycina',
  'fccpVec',
  'fcppVec',
  'oligomycin_vector',
  'rotenone_antimycina_vector']}

In [124]:
# calculate p-value and adjusted p-value for each row 
import scipy.stats as stats
for i, row in volcano_df.iterrows():
    treatment = row['treatment']
    cell_line = row['cell_line']
    experiment = row['experiment']
    match = df[(df['treatment'] == treatment) & (df['cell_line'] == cell_line) & (df['experiment'] == experiment)]
    if experiment == 'confluency':
        control_match = df[(df['treatment'] == 'C-09') & (df['cell_line'] == cell_line) & (df['experiment'] == experiment)]
    else:
        control_match = df[(df['treatment'] == '0-control') & (df['cell_line'] == cell_line) & (df['experiment'] == experiment)]
    num_comparisons = len(experiments_treatment_dict[experiment])
    for measure in measure_names:
        if len(control_match) == len(match): 
            t_statistic, p_value = stats.ttest_ind(np.float64(match[measure]), np.float64(control_match[measure]), equal_var=True)
        else: t_statistic, p_value = stats.ttest_ind(np.float64(match[measure]), np.float64(control_match[measure]), equal_var=False)
        if p_value < 10**-10:
            p_value = 10**-10
        volcano_df.at[i, measure + "_pvalue"] = p_value
        # perform Bonferroni Correction 
        volcano_df.at[i, measure + "_pvadj"] = p_value * num_comparisons
        

In [125]:
volcano_df.to_csv("volcano.csv", index=False)

In [110]:
print(np.finfo(np.float64).tiny)

2.2250738585072014e-308
